In [1]:
from classify_abs import search_Pubmed_API, search_EBI_API
from extract_abs import autosearch, load_GARD_diseases
import pandas as pd
import csv

GARD_dict, max_length = load_GARD_diseases()

In [2]:
searchlist = ['Morphea','Sandhoff disease','Santos Mateus Leal syndrome','Facioscapulohumeral muscular dystrophy','Fellman syndrome','Mucopolysaccharidosis type 6','Neurofibromatosis type 1','homocystinuria','0007730']

In [3]:
for searching in searchlist:
    PubMedAPI = search_Pubmed_API(autosearch(searching,GARD_dict), 50)
    EBIAPI = search_EBI_API(autosearch(searching,GARD_dict), 50)

    intersect = set(EBIAPI.keys()).intersection(set(PubMedAPI.keys()))
    U = set(EBIAPI.keys()).union(set(PubMedAPI.keys()))

    print('EBI Result',len(EBIAPI.keys()))
    print('PubMed Result',len(PubMedAPI.keys()))
    print(searching,'intersection: ',len(intersect),', union: ',len(U))
    print(searching,100*len(intersect)/len(U),'% intersection')
    print('')
    
    #SAVE RESULTS
    # csv header
    header = ['EBI API PMIDS', 'PubMed API PMIDS']
    PM_list = list(PubMedAPI.keys())
    EBI_list = list(EBIAPI.keys())

    # csv data
    rows = [EBIAPI,PubMedAPI]

    max_l = len(PM_list)
    if len(EBI_list)>max_l:
        max_l = len(EBI_list)

    import csv
    with open(str('API-Analysis/'+searching+' API Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for i in range(max_l):
            EBI_fill = ''
            PM_fill = ''
            if i<len(PM_list):
                PM_fill = PM_list[i]
            if i<len(EBI_list):
                EBI_fill = EBI_list[i]
            writer.writerow([EBI_fill,PM_fill])

SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['morphea']
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['morphea']
EBI Result 15
PubMed Result 14
Morphea intersection:  9 , union:  20
Morphea 45.0 % intersection

SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['sandhoff disease', 'beta-hexosaminidase-beta-subunit deficiency', 'gm2 gangliosidosis, type 2', 'total hexosaminidase deficiency', 'hexosaminidase a and b deficiency disease', 'sandhoff-jatzkewitz-pilz disease']
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['sandhoff disease', 'beta-hexosaminidase-beta-subunit deficiency', 'gm2 gangliosidosis, type 2', 'total hexosaminidase deficiency', 'hexosaminidase a and b deficiency disease', 'sandhoff-jatzkewitz-pilz disease']
EBI Result 45
PubMed Result 21
Sandhoff disease intersection:  8 , union:  58
Sandhoff disease 13.793103448275861 % intersection

SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['santos mateus leal syndrome', 'hirschs

Also get all of the Orphanet data in a readable and searchable format

In [4]:
import xml.etree.ElementTree as ET
import re
import pandas as pd

In [5]:
#This file was downloaded on August 31, 2021. See README.md for details
tree = ET.parse('en_product9_prev.xml')
root = tree.getroot()

In [6]:
df = pd.DataFrame(columns=['OrphaCode',
                           'Disease Name',
                           'Orpha Epi Type',
                           'Orpha Epi Class',
                           'Orpha Epi Rate',
                           'Orpha Loc',
                           'Source',
                           'PMID'])

In [7]:
i=0
for disorder in root.iter('Disorder'):  
    code = disorder.find('./OrphaCode').text
    name = disorder.find('./Name').text
    #Each disorder, w/code and name, has multiple prevalence branches
    for prevalence in disorder.findall('./PrevalenceList/Prevalence'):
        EPtype = prevalence.find('./PrevalenceType/Name').text
        if 'class' in prevalence.find('./PrevalenceQualification/Name').text.lower():
            EPclss = prevalence.find('./PrevalenceClass/Name').text
        else:
            EPclss = ''
        EPrate = prevalence.find('./ValMoy').text
        geoloc = prevalence.find('./PrevalenceGeographic/Name').text
        source = prevalence.find('./Source').text
        #each prevalence, w/geoloc and source, has multiple pmids w/abstracts
        if 'PMID' in str(source) and len(EPclss)>1:
            pmids = re.findall('\d{6,8}', source)
            for articleid in pmids:
                pmid = articleid
                df = df.append({'OrphaCode':code,
                                    'Disease Name':name,
                                    'Orpha Epi Type':EPtype,
                                    'Orpha Epi Class':EPclss,
                                    'Orpha Epi Rate':EPrate,
                                    'Orpha Loc':geoloc,
                                    'Source':source,
                                    'PMID':pmid}
                                   , ignore_index=True)
                i+=1
                if i%500==0:
                    print(i)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


In [8]:
df

,OrphaCode,Disease Name,Orpha Epi Type,Orpha Epi Class,Orpha Epi Rate,Orpha Loc,Source,PMID
0,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",Point prevalence,<1 / 1 000 000,0.0,Worldwide,ORPHANET_11389160[PMID]_9689990[PMID],11389160
1,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",Point prevalence,<1 / 1 000 000,0.0,Worldwide,ORPHANET_11389160[PMID]_9689990[PMID],9689990
2,58,Alexander disease,Annual incidence,<1 / 1 000 000,0.037,Japan,21533827[PMID]_[EXPERT],21533827
3,166032,"Multiple epiphyseal dysplasia, with miniepiphyses",Point prevalence,<1 / 1 000 000,0.0,Worldwide,ORPHANET_15523498[PMID],15523498
4,61,Alpha-mannosidosis,Prevalence at birth,<1 / 1 000 000,0.09,Australia,9918480[PMID],9918480
...,...,...,...,...,...,...,...,...
7202,99807,PEHO-like syndrome,Point prevalence,<1 / 1 000 000,0.0,Worldwide,ORPHANET_15968934[PMID]_22408680[PMID],15968934
7203,99807,PEHO-like syndrome,Point prevalence,<1 / 1 000 000,0.0,Worldwide,ORPHANET_15968934[PMID]_22408680[PMID],22408680
7204,99789,Dentin dysplasia type I,Point prevalence,1-9 / 100 000,1.0,Europe,17452557[PMID]_[EXPERT],17452557
7205,99792,Dentin dysplasia-sclerotic bones syndrome,Point prevalence,<1 / 1 000 000,0.0,Worldwide,ORPHANET_264650[PMID],264650


In [9]:
#Not going to remove the duplicate PMIDs because Orphanet has different stat, disease, loc, info for each duplicated entry.
df.to_csv('All Orphanet Data.csv', header = True,index=False, encoding='utf-8')
print('DONE')

DONE
